#**스마트폰 센서 데이터 기반 모션 분류**
# 단계2 : 기본 모델링


## 0.미션

* 데이터 전처리
    * 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리 수행
* 다양한 알고리즘으로 분류 모델 생성
    * 최소 4개 이상의 알고리즘을 적용하여 모델링 수행
    * 성능 비교
    * 각 모델의 성능을 저장하는 별도 데이터 프레임을 만들고 비교
* 옵션 : 다음 사항은 선택사항입니다. 시간이 허용하는 범위 내에서 수행하세요.
    * 상위 N개 변수를 선정하여 모델링 및 성능 비교
        * 모델링에 항상 모든 변수가 필요한 것은 아닙니다.
        * 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교하세요.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것입니다.

## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.




* 함수 생성

In [ ]:
# 변수의 특성 중요도 계산하기
def plot_feature_importance(importance, names, result_only = False, topn = 'all'):
    feature_importance = np.array(importance)
    feature_name = np.array(names)

    data={'feature_name':feature_name,'feature_importance':feature_importance}
    fi_temp = pd.DataFrame(data)

    #변수의 특성 중요도 순으로 정렬하기
    fi_temp.sort_values(by=['feature_importance'], ascending=False,inplace=True)
    fi_temp.reset_index(drop=True, inplace = True)

    if topn == 'all' :
        fi_df = fi_temp.copy()
    else :
        fi_df = fi_temp.iloc[:topn]

    #변수의 특성 중요도 그래프로 그리기
    if result_only == False :
        plt.figure(figsize=(10,20))
        sns.barplot(x='feature_importance', y='feature_name', data = fi_df)

        plt.xlabel('importance')
        plt.ylabel('feature name')
        plt.grid()

    return fi_df

### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용
* 세부 요구사항
    - 전체 데이터 'data01_train.csv' 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - 데이터프레임에 대한 기본 정보를 확인합니다.( .head(), .shape 등)

#### 1) 데이터 로딩

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/2023.04.12_미니프로젝트5차_3_5일차 실습자료/data01_train.csv')

In [ ]:
data.isnull().sum()

tBodyAcc-mean()-X       0
tBodyAcc-mean()-Y       0
tBodyAcc-mean()-Z       0
tBodyAcc-std()-X        0
tBodyAcc-std()-Y        0
                       ..
angle(X,gravityMean)    0
angle(Y,gravityMean)    0
angle(Z,gravityMean)    0
subject                 0
Activity                0
Length: 563, dtype: int64

In [ ]:
data.drop('subject',axis=1,inplace=True)

#### 2) 기본 정보 조회

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5881 entries, 0 to 5880
Columns: 562 entries, tBodyAcc-mean()-X to Activity
dtypes: float64(561), object(1)
memory usage: 25.2+ MB


## **2. 데이터 전처리**

* 가변수화, 데이터 분할, NaN 확인 및 조치, 스케일링 등 필요한 전처리를 수행한다.


### (1) 데이터 분할1 : x, y

* 세부 요구사항
    - x, y로 분할합니다.

In [ ]:
target = 'Activity'
x =data.drop(target,axis=1)
y=data.loc[:,target]

In [ ]:
y

0                 STANDING
1                   LAYING
2                 STANDING
3                  WALKING
4       WALKING_DOWNSTAIRS
               ...        
5876               SITTING
5877      WALKING_UPSTAIRS
5878                LAYING
5879      WALKING_UPSTAIRS
5880               SITTING
Name: Activity, Length: 5881, dtype: object

### (2) 스케일링(필요시)


* 세부 요구사항
    - 스케일링을 필요로 하는 알고리즘 사용을 위해서 코드 수행
    - min-max 방식 혹은 standard 방식 중 한가지 사용.

### (3) 데이터분할2 : train, validation

* 세부 요구사항
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=1)

In [ ]:
!pip install pycaret

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.8/481.8 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 39.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 7.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.8/105.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.2/81.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 80.6 MB/s eta 0:00:00


In [ ]:
from pycaret.classification import *

## **3. 기본 모델링**



* 세부 요구사항
    - 최소 4개 이상의 알고리즘을 적용하여 모델링을 수행한다.
    - 각 알고리즘별로 전체 변수로 모델링, 상위 N개 변수를 선택하여 모델링을 수행하고 성능 비교를 한다.
    - (옵션) 알고리즘 중 1~2개에 대해서, 변수 중요도 상위 N개를 선정하여 모델링하고 타 모델과 성능을 비교.
        * 상위 N개를 선택하는 방법은, 변수를 하나씩 늘려가며 모델링 및 성능 검증을 수행하여 적절한 지점을 찾는 것이다.

### (1) 알고리즘1 :

In [ ]:
exp_clf101 = setup(data=data,
                   target=target,
                   fold=5,
                   session_id=123)

,Description,Value
0,Session id,123
1,Target,Activity
2,Target type,Multiclass
3,Target mapping,"LAYING: 0, SITTING: 1, STANDING: 2, WALKING: 3, WALKING_DOWNSTAIRS: 4, WALKING_UPSTAIRS: 5"
4,Original data shape,"(5881, 562)"
5,Transformed data shape,"(5881, 562)"
6,Transformed train set shape,"(4116, 562)"
7,Transformed test set shape,"(1765, 562)"
8,Numeric features,561
9,Preprocess,True


In [ ]:
# 1) 이미 존재하는 metric을 가져와 추가하는 경우
# sklearn에서 제공하는 metric인 log loss를 평가지표로 추가
from sklearn.metrics import log_loss
add_metric('logloss',               # id
           'Log Loss',              # name
            log_loss,               # 함수
            greater_is_better=False # 함수 결과값이 커야 좋은가?
            )

Name                                                       Log Loss
Display Name                                               Log Loss
Score Function                <function log_loss at 0x7efe79e70ca0>
Scorer               make_scorer(log_loss, greater_is_better=False)
Target                                                         pred
Args                                                             {}
Greater is Better                                             False
Multiclass                                                     True
Custom                                                         True
Name: logloss, dtype: object

In [ ]:
get_metrics() # 사용될 평가지표 확인

,Name,Display Name,Score Function,Scorer,Target,Args,Greater is Better,Multiclass,Custom
ID,,,,,,,,,
acc,Accuracy,Accuracy,<function accuracy_score at 0x7efe79e70280>,accuracy,pred,{},True,True,False
auc,AUC,AUC,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(roc_auc_score, needs_proba=True, e...",pred_proba,"{'average': 'weighted', 'multi_class': 'ovr'}",True,True,False
recall,Recall,Recall,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(recall_score, average=weighted, po...",pred,"{'average': 'weighted', 'pos_label': 'WALKING_...",True,True,False
precision,Precision,Prec.,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(precision_score, average=weighted,...",pred,"{'average': 'weighted', 'pos_label': 'WALKING_...",True,True,False
f1,F1,F1,<pycaret.internal.metrics.BinaryMulticlassScor...,"make_scorer(f1_score, average=weighted, pos_la...",pred,"{'average': 'weighted', 'pos_label': 'WALKING_...",True,True,False
kappa,Kappa,Kappa,<function cohen_kappa_score at 0x7efe79e703a0>,make_scorer(cohen_kappa_score),pred,{},True,True,False
mcc,MCC,MCC,<function matthews_corrcoef at 0x7efe79e704c0>,make_scorer(matthews_corrcoef),pred,{},True,True,False
logloss,Log Loss,Log Loss,<function log_loss at 0x7efe79e70ca0>,"make_scorer(log_loss, greater_is_better=False)",pred,{},False,True,True


In [ ]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Log Loss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9871,0.9998,0.9871,0.9872,0.9871,0.9845,0.9845,0.0000,25.7820
gbc,Gradient Boosting Classifier,0.9837,0.9995,0.9837,0.9838,0.9837,0.9804,0.9804,0.0000,72.7660
lr,Logistic Regression,0.9832,0.9992,0.9832,0.9833,0.9832,0.9798,0.9798,0.0000,0.7720
xgboost,Extreme Gradient Boosting,0.9815,0.9996,0.9815,0.9817,0.9815,0.9778,0.9778,0.0000,44.7820
et,Extra Trees Classifier,0.9813,0.9994,0.9813,0.9814,0.9813,0.9775,0.9775,0.0000,1.3720
ridge,Ridge Classifier,0.9796,0.0000,0.9796,0.9797,0.9796,0.9754,0.9755,0.0000,0.1820
lda,Linear Discriminant Analysis,0.9786,0.9990,0.9786,0.9787,0.9786,0.9743,0.9743,0.0000,0.9260
rf,Random Forest Classifier,0.9694,0.9992,0.9694,0.9696,0.9694,0.9632,0.9632,0.0000,0.3900
svm,SVM - Linear Kernel,0.9667,0.0000,0.9667,0.9706,0.9665,0.9599,0.9609,0.0000,0.4260
knn,K Neighbors Classifier,0.9514,0.9943,0.9514,0.9521,0.9512,0.9415,0.9418,0.0000,0.4800


Processing:   0%|          | 0/65 [00:00<?, ?it/s]

In [ ]:
result = pull()
display(result)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,Log Loss,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.9871,0.9998,0.9871,0.9872,0.9871,0.9845,0.9845,0.0,25.782
gbc,Gradient Boosting Classifier,0.9837,0.9995,0.9837,0.9838,0.9837,0.9804,0.9804,0.0,72.766
lr,Logistic Regression,0.9832,0.9992,0.9832,0.9833,0.9832,0.9798,0.9798,0.0,0.772
xgboost,Extreme Gradient Boosting,0.9815,0.9996,0.9815,0.9817,0.9815,0.9778,0.9778,0.0,44.782
et,Extra Trees Classifier,0.9813,0.9994,0.9813,0.9814,0.9813,0.9775,0.9775,0.0,1.372
ridge,Ridge Classifier,0.9796,0.0000,0.9796,0.9797,0.9796,0.9754,0.9755,0.0,0.182
lda,Linear Discriminant Analysis,0.9786,0.9990,0.9786,0.9787,0.9786,0.9743,0.9743,0.0,0.926
rf,Random Forest Classifier,0.9694,0.9992,0.9694,0.9696,0.9694,0.9632,0.9632,0.0,0.390
svm,SVM - Linear Kernel,0.9667,0.0000,0.9667,0.9706,0.9665,0.9599,0.9609,0.0,0.426
knn,K Neighbors Classifier,0.9514,0.9943,0.9514,0.9521,0.9512,0.9415,0.9418,0.0,0.480
